# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data.sqlite`.

In [1]:
# Your code here; import the necessary packages
import pandas as pd
import sqlite3

In [2]:
# Your code here; create the connection
conn = sqlite3.connect('data.sqlite')

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [3]:
# Your code here
q = """
SELECT c.customerNumber, contactLastName, contactFirstName
  FROM customers c
 WHERE c.customerNumber IN (SELECT o.customerNumber
                             FrOM orders o
                            WHERE orderDate = '2003-01-31')
;
"""

df = pd.read_sql(q, conn)
print("Number of records:", len(df))
df.head()

Number of records: 1


,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [4]:
# Your code here
q = """
SELECT productName, COUNT(orderNumber) AS num_orders, SUM(quantityOrdered) AS total_ordered
  FROM products
  JOIN orderdetails
     USING(productCode)
 GROUP BY productName
 ORDER BY total_ordered DESC
;
"""

df = pd.read_sql(q, conn)
print("Number of records:", len(df))
df.head()

Number of records: 109


,productName,num_orders,total_ordered
0,1992 Ferrari 360 Spider red,53,1808
1,1937 Lincoln Berline,28,1111
2,American Airlines: MD-11S,28,1085
3,1941 Chevrolet Special Deluxe Cabriolet,28,1076
4,1930 Buick Marquette Phaeton,28,1074


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [5]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
q = """
SELECT productName, COUNT(DISTINCT(customerNumber)) AS numPeople
  FROM products
  JOIN orderdetails
     USING(productCode)
  JOIN orders
     USING(orderNumber)
 GROUP BY productName
 ORDER BY numPeople DESC
;
"""

pd.read_sql(q, conn)

,productName,numPeople
0,1992 Ferrari 360 Spider red,40
1,Boeing X-32A JSF,27
2,1972 Alfa Romeo GTA,27
3,1952 Alpine Renault 1300,27
4,1934 Ford V8 Coupe,27
...,...,...
104,1958 Chevy Corvette Limited Edition,19
105,2002 Chevy Corvette,18
106,1969 Chevrolet Camaro Z28,18
107,1952 Citroen-15CV,18


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [ ]:
SELECT employeeNumber, firstName, lastName, city, officeCode
  FROM employees
  JOIN offices
     USING(officeCode)

In [20]:
# Your code here

q = """
SELECT DISTINCT e.employeeNumber, e.firstName, e.lastName, e.officeCode, o.city
  FROM employees e
  JOIN offices o
     USING(officeCode)
  JOIN customers c
     ON e.employeeNumber = c.salesRepEmployeeNumber
  JOIN orders
     USING(customerNumber)
  JOIN orderdetails
     USING(orderNumber)
 WHERE productCode IN (
        SELECT productCodE
          FROM products
          JOIN orderdetails
             USING(productCode)
          JOIN orders
             USING(orderNumber)
         GROUP BY productName
         HAVING COUNT(DISTINCT(customerNumber)) < 20)
;
"""

pd.read_sql(q, conn)

,employeeNumber,firstName,lastName,officeCode,city
0,1370,Gerard,Hernandez,4,Paris
1,1501,Larry,Bott,7,London
2,1337,Loui,Bondur,4,Paris
3,1166,Leslie,Thompson,1,San Francisco
4,1286,Foon Yue,Tseng,3,NYC
5,1612,Peter,Marsh,6,Sydney
6,1611,Andy,Fixter,6,Sydney
7,1401,Pamela,Castillo,4,Paris
8,1621,Mami,Nishi,5,Tokyo
9,1323,George,Vanauf,3,NYC


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [35]:
# Your code here
q = """
SELECT DISTINCT employeeNumber, firstName, lastName, COUNT(customerNumber)
  FROM employees e
  JOIN customers c
     ON e.employeeNumber = c.SalesRepEmployeeNumber
  GROUP BY employeeNumber
 HAVING e.employeeNumber IN (
        SELECT salesRepEmployeeNumber
          FROM customers
         GROUP BY salesRepEmployeeNumber
        HAVING AVG(creditLimit) > 15000)
;
"""

pd.read_sql(q, conn)

,employeeNumber,firstName,lastName,COUNT(customerNumber)
0,1165,Leslie,Jennings,6
1,1166,Leslie,Thompson,6
2,1188,Julie,Firrelli,6
3,1216,Steve,Patterson,6
4,1286,Foon Yue,Tseng,7
5,1323,George,Vanauf,8
6,1337,Loui,Bondur,6
7,1370,Gerard,Hernandez,7
8,1401,Pamela,Castillo,10
9,1501,Larry,Bott,8


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!